In [1]:
import sys
import os

if "../" not in sys.path:
    sys.path.append("../")

In [2]:
# import glob
# import shutil

# # Get a list of all CSV files in the source directory
# csv_files = glob.glob("../datasets/*.csv")

# # Move each CSV file to the destination directory
# for file in csv_files:
#     shutil.move(file, "../results/")

In [3]:
from jobshop.base import JobShopBase
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
from collections import namedtuple, defaultdict
from typing import List, Tuple, Set


class LPJS:
    def __init__(
        self,
        jobs_data: List[List[Tuple[int, float]]],
        pre: List[List[Tuple[int, int]]],
        capacities: List[List[int]],
        cost: List[float],
    ):
        self.jobs_data = jobs_data
        self.pre = pre
        self.capacities = capacities
        self.cost = cost
        self.n_jobs = len(jobs_data)
        self.n_machines = len(cost)
        self.horizon = sum(
            [sum([duration for _, duration in job]) for job in jobs_data]
        )
        self.model = pywraplp.Solver("jobshop", pywraplp.Solver.CLP_LINEAR_PROGRAMMING)

    def addConstraints(self):
        # task = namedtuple("task", "start end interval")

        #         """
        #         Ràng buộc thời gian bắt đầu và kết thúc của mỗi task.
        #         Điều kiện :
        #             - Thời gian bắt đầu và kết thúc của mỗi task phải thỏa mãn điều kiện : 0 <= start <= end <= horizon
        #             -  start + duration = end
        #         """
        start_vars = [defaultdict() for _ in range(self.n_jobs)]
        end_vars = [defaultdict() for _ in range(self.n_jobs)]
        for job, task in enumerate(self.jobs_data):
            for task_type, processing_time in task:
                start_vars[job][task_type] = self.model.NumVar(
                    0, self.horizon, "start_%i_%i" % (job, task_type)
                )
                end_vars[job][task_type] = self.model.NumVar(
                    0, self.horizon, "end_%i_%i" % (job, task_type)
                )
                self.model.Add(
                    end_vars[job][task_type]
                    == start_vars[job][task_type] + processing_time
                )
        for job, task in enumerate(self.pre):
            for task1, task2 in task:
                self.model.Add(end_vars[job][task1] <= start_vars[job][task2])

        # Assign tasks to machines
        task_to_machine = defaultdict(list)
        machine_tasks = defaultdict(list)
        for job, task in enumerate(self.jobs_data):
            for task_type, _ in task:
                for machine, capacity in enumerate(self.capacities):
                    capacity = set(capacity)
                    if task_type in capacity:
                        decision_var = self.model.BoolVar(
                            "job_%i_task_%i_machine_%i" % (job, task_type, machine)
                        )
                        task_to_machine[(job, task_type)].append(decision_var)
                        machine_tasks[machine].append(decision_var)
                self.model.Add(sum(task_to_machine[(job, task_type)]) == 1)
        # self.model.Add

#             for j in self.all_machines:
#                 start_var = self.model.NewIntVar(
#                     0, self.horizon, "start_%i_%i" % (i, j)
#                 )
#                 duration = self.durations[job][j]
#                 end_var = self.model.NewIntVar(0, self.horizon, "end_%i_%i" % (i, j))
#                 interval_var = self.model.NewIntervalVar(
#                     start_var, duration, end_var, "interval_%i_%i" % (i, j)
#                 )
#                 self.all_tasks[(i, j)] = task_type(
#                     start=start_var, end=end_var, interval=interval_var
#                 )

#         """
#         Ràng buộc mỗi máy chỉ được thực hiện một task tại một thời điểm.
#         """
#         machine_to_jobs = {}
#         for i in self.all_machines:
#             machines_jobs = []
#             for j in self.all_jobs:
#                 for k in self.all_machines:
#                     if (
#                         self.machines[j][k] == i
#                     ):  # Tìm tất cả các task cần thực hiện trên máy i
#                         machines_jobs.append(self.all_tasks[(j, k)].interval)
#             machine_to_jobs[job] = machines_jobs
#             # Mỗi máy chỉ được thực hiện một task tại một thời điểm (các khoảng thời gian không được chồng lấn lên nhau)
#             self.model.AddNoOverlap(machines_jobs)

#         """
#         Ràng buộc tuần tự công nghệ:
#             Với mỗi task của job, thời gian bắt đầu của task tiếp theo phải >= thời gian kết thúc của task trước đó.
#         """
#         for i in self.all_jobs:
#             for j in range(0, self.n_machines - 1):
#                 self.model.Add(
#                     self.all_tasks[(i, j + 1)].start >= self.all_tasks[(i, j)].end
#                 )

#         """
#         Mục tiêu cần tối ưu:
#             Tìm thời gian hoàn thành của job cuối cùng (makespan) là nhỏ nhất.
#         """
#         self.obj_var = self.model.NewIntVar(0, self.horizon, "makespan")
#         self.model.AddMaxEquality(
#             self.obj_var,
#             [self.all_tasks[(i, self.n_machines - 1)].end for i in self.all_jobs],
#         )  # Thời gian lớn nhất để hoàn thành task cuối cùng của tất cả các job chính là thời gian hoàn thành tất cả các jobs (makespan)
#         self.model.Minimize(self.obj_var)

#     def solve(self, max_time_in_seconds=60.0, display=False):
#         self.solver = cp_model.CpSolver()
#         self.solver.parameters.log_search_progress = True
#         self.solver.parameters.max_time_in_seconds = max_time_in_seconds
#         status = self.solver.Solve(self.model)
#         stt = None
#         if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
#             stt = self.solver.StatusName(status)
#             self.makespan = self.solver.ObjectiveValue()
#             self.starts = [
#                 [
#                     self.solver.Value(self.all_tasks[(i, j)][0])
#                     for j in self.all_machines
#                 ]
#                 for i in self.all_jobs
#             ]
#         # # Print sequences of jobs assigned to each machine.
#         # starts = [
#         #     [(starts[job][j], machines[job][j]) for j in all_machines] for i in all_jobs
#         # ]
#         # starts = [sorted(job, key=lambda x: x[1]) for job in starts]
#         # job_sq = []
#         # for start in zip(*starts):
#         #     start_job = sorted(
#         #         [(i, j[0]) for i, j in enumerate(start)], key=lambda x: x[1]
#         #     )
#         #     job_sq.append([i[0] for i in start_job])
#         # print()
#         # print("Optimal job sequence: ")
#         # for i in job_sq:
#         #     print(*i)
#         else:
#             stt = self.solver.StatusName(status)
#         if display:
#             self.display(stt, self.starts, self.makespan)
#         return self.makespan

#     def summary(self):
#         print("Constraint programming model summary: ")
#         print(self.solver.ResponseStats())
#         print(self.solver.SolutionInfo())

In [4]:
jobs_data = [[(0, 3), (1, 2.5), (2, 2)], [
    (0, 2), (2, 1), (1, 4)], [(1, 4), (2, 3)]]
pre = [[(0, 2), (2, 1)], [], [(1, 2)]]
capacities = [[0, 1], [0, 1, 2], [1]]
cost = [4, 3.5, 6]
model = LPJS(jobs_data, pre, capacities, cost)
model.addConstraints()